In [75]:
import bw2data as bd
import bw2io as bi
import bw2calc as bc
import pandas as pd
from scipy.special import expit
from matplotlib import pyplot as plt
import numpy as np
import bw_processing as bwp
from carculator import *

from bw_processing import load_datapackage
from fs.zipfs import ZipFS

import pandas as pd


In [246]:
def replacement(steel_am):
    return (1300 - steel_am)/2

In [330]:
class MyCoolAluminiumThing:
    def __next__(self):
        steel_new = 1300 - np.random.randint(50,300)
        alu = replacement(steel_new)
        return np.array([steel_new, alu])

# Ecoinvent importer 

In [331]:
interf = MyCoolAluminiumThing()
next(interf)

array([1212.,   44.])

bd.projects.delete_project('project')

In [4]:
bd.projects.set_current("new_test2")

In [5]:
bi.databases

Databases dictionary with 0 objects

In [6]:
if 'biosphere3' in bd.databases:
    del bd.databases['biosphere3']
    
if 'ecoinvent 3.7 cutoff' in bd.databases:
    del bd.databases['ecoinvent 3.7 cutoff']

In [7]:
bi.bw2setup()

Creating default biosphere



Writing activities to SQLite3 database:


Applying strategy: normalize_units
Applying strategy: drop_unspecified_subcategories
Applying strategy: ensure_categories_are_tuples
Applied 3 strategies in 0.01 seconds


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 10/21/2021 11:38:41
  Finished: 10/21/2021 11:38:41
  Total time elapsed: 00:00:00
  CPU %: 93.70
  Memory %: 1.18
Created database: biosphere3
Creating default LCIA methods

Applying strategy: normalize_units
Applying strategy: set_biosphere_type
Applying strategy: fix_ecoinvent_38_lcia_implementation
Applying strategy: drop_unspecified_subcategories
Applying strategy: link_iterable_by_fields
Applied 5 strategies in 2.38 seconds
Wrote 975 LCIA methods with 254388 characterization factors
Creating core data migrations



In [8]:
ei = bi.SingleOutputEcospold2Importer(r"C:/Users/lorenzou/Box Sync/PhD work/Brightway/ei/datasets/",
                                           'ecoinvent 3.7 cutoff'
                                          )
ei.apply_strategies()
ei.statistics()

Extracting XML data from 19128 datasets
Extracted 19128 datasets in 146.94 seconds
Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: remove_zero_amount_coproducts
Applying strategy: remove_zero_amount_inputs_with_no_activity
Applying strategy: remove_unnamed_parameters
Applying strategy: es2_assign_only_product_with_amount_as_reference_product
Applying strategy: assign_single_product_as_activity
Applying strategy: create_composite_code
Applying strategy: drop_unspecified_subcategories
Applying strategy: fix_ecoinvent_flows_pre35
Applying strategy: drop_temporary_outdated_biosphere_flows
Applying strategy: link_biosphere_by_flow_uuid
Applying strategy: link_internal_technosphere_by_composite_code
Applying strategy: delete_exchanges_missing_activity
1 exchanges couldn't be linked and were deleted. See the logfile for details:
	C:\Users\lorenzou\AppData\Local\pylca\Brightway3\Logs\new_test2.d76c312f\Ecospold2-import-error.SlsiLR.log
Applyi

(19128, 621717, 0)

In [14]:
ei.all_linked

AttributeError: 'SQLiteBackend' object has no attribute 'all_linked'

In [15]:
ei.write_database()

AttributeError: 'SQLiteBackend' object has no attribute 'write_database'

In [208]:
biosphere = bd.Database('biosphere3')
biosphere.register()

In [17]:
ei = bd.Database('ecoinvent 3.7 cutoff')
ei.register()

# Read data from carculator - LOL

In [20]:
ele = ei.search('market for electricity, low-voltage RER')

In [21]:
for act in [act for act in ei if 'market for steel,' in act['name']]:
    print(act)

'market for steel, unalloyed' (kilogram, GLO, None)
'market for steel, chromium steel 18/8, hot rolled' (kilogram, GLO, None)
'market for steel, low-alloyed, hot rolled' (kilogram, GLO, None)
'market for steel, low-alloyed' (kilogram, GLO, None)
'market for steel, chromium steel 18/8' (kilogram, GLO, None)


In [22]:
for act in [act for act in ei if 'aluminium' in act['name']]:
    print(act)

'aluminium oxide production' (kilogram, CN, None)
'anode production, paste, for aluminium electrolysis' (kilogram, IAI Area, Russia & RER w/o EU27 & EFTA, None)
'aluminium hydroxide production' (kilogram, RNA, None)
'electricity production, coal, aluminium industry' (kilowatt hour, UN-OCEANIA, None)
'treatment of aluminium scrap, post-consumer, prepared for recycling, at refiner' (kilogram, RoW, None)
'aluminium scrap, post-consumer, prepared for melting, Recycled Content cut-off' (kilogram, GLO, None)
'market for cladding, crossbar-pole, aluminium' (square meter, GLO, None)
'aluminium milling, average' (kilogram, RER, None)
'aluminium around steel bi-metal stranded cable production, 3x3.67mm external diameter wire' (meter, RoW, None)
'door production, outer, wood-aluminium' (square meter, RoW, None)
'aluminium oxide production' (kilogram, UN-OCEANIA, None)
'market for scrap aluminium' (kilogram, CH, None)
'aluminium oxide factory construction' (unit, RER, None)
'aluminium production, 

In [23]:
for activity in [act for act in ei if 'Glider' in act['name'] or 'ICEV_prod' in act['name'] or 'EV_prod' in act['name']]:
    activity.delete()
    

    
EV_prod = ei.new_activity(code = 'electric_vehicles', name = 'EV_prod', unit = 'unit')
EV_prod.save()

ICEV_prod = ei.new_activity(code = 'icev', name = 'ICEV_prod', unit = 'unit')
ICEV_prod.save()
    
glider = ei.new_activity(code = 'test_glider', name = 'Glider', unit = 'unit')
glider.save()

In [24]:
steel = [act for act in ei if act['name']== 'market for steel, low-alloyed'][0]
aluminium = [act for act in ei if act['name'] == 'aluminium alloy production, AlMg3'][0]
carbon_fibre = [act for act in ei if act['name'] == 'market for carbon fibre reinforced plastic, injection moulded'][0]


In [85]:
steel_amount = 1300
aluminium_amount = 200

In [25]:
glider.new_exchange(input = steel.key, amount = steel_amount, unit = 'kg', type = 'technosphere').save()
glider.new_exchange(input = aluminium.key, amount = aluminium_amount, unit = 'kg', type = 'technosphere').save()


In [26]:
for exc in glider.exchanges():
    print(exc)

Exchange: 1300 kilogram 'market for steel, low-alloyed' (kilogram, GLO, None) to 'Glider' (unit, GLO, None)>
Exchange: 200 kilogram 'aluminium alloy production, AlMg3' (kilogram, RER, None) to 'Glider' (unit, GLO, None)>
Exchange: 0 kilogram 'market for carbon fibre reinforced plastic, injection moulded' (kilogram, GLO, None) to 'Glider' (unit, GLO, None)>


In [27]:
ICEV_prod.new_exchange(input = glider, amount = 1, unit = 'unit', type = 'technosphere').save()

EV_prod.new_exchange(input = glider, amount = 1.1, unit = 'unit', type = 'technosphere').save()

In [28]:
for act in [act for act in ei if 'battery cell production' in act['name'] and 'RoW' in act['location']]:
    print(act)

'battery cell production, Li-ion' (kilogram, RoW, None)


In [29]:
LIB = [act for act in ei if 'battery cell production' in act['name'] and 'RoW' in act['location']][0]

In [30]:
EV_prod.new_exchange(input = LIB, amount = 340, unit = 'kg', type = 'technosphere').save()

In [31]:
for exc in EV_prod.exchanges():
    print(exc)

Exchange: 1.1 unit 'Glider' (unit, GLO, None) to 'EV_prod' (unit, GLO, None)>
Exchange: 340 kilogram 'battery cell production, Li-ion' (kilogram, RoW, None) to 'EV_prod' (unit, GLO, None)>


In [32]:
for activity in [act for act in ei if 'ICEV_tot' in act['name'] or 'EV_tot' in act['name']]:
    activity.delete()
    

EV_tot = ei.new_activity(code = 'ev-tot', name ='EV_tot', unit = 'unit/vkm')
EV_tot.save()

ICEV_tot = ei.new_activity(code = 'icev-tot', name = 'ICEV_tot', unit = 'unit/vkm')
ICEV_tot.save()

In [33]:
EV_tot.new_exchange(input = EV_prod, amount = 1/150000, type = 'technosphere').save()
ICEV_tot.new_exchange(input = ICEV_prod, amount = 1/150000, type = 'technosphere').save()

In [34]:
for exc in EV_tot.exchanges():
    print(exc)

Exchange: 6.666666666666667e-06 unit 'EV_prod' (unit, GLO, None) to 'EV_tot' (unit/vkm, GLO, None)>


In [35]:
for act in [act for act in ei if 'petrol production' in act['name']]:
    print(act)

'petrol production, 4% ETBE additive by volume, with ethanol from biomass' (kilogram, RER, None)
'petrol production, unleaded, petroleum refinery operation' (kilogram, PE, None)
'petrol production, 5% ethanol by volume from biomass' (kilogram, CH, None)
'petrol production, 15% ETBE additive by volume, with ethanol from biomass' (kilogram, RoW, None)
'petrol production, 15% ETBE additive by volume, with ethanol from biomass' (kilogram, RER, None)
'petrol production, 85% ethanol by volume from biomass' (kilogram, CH, None)
'petrol production, low-sulfur' (kilogram, CH, None)
'petrol production, unleaded, petroleum refinery operation' (kilogram, CO, None)
'petrol production, unleaded, petroleum refinery operation' (kilogram, Europe without Switzerland, None)
'petrol production, unleaded, petroleum refinery operation' (kilogram, RoW, None)
'petrol production, low-sulfur' (kilogram, Europe without Switzerland, None)
'petrol production, unleaded, petroleum refinery operation' (kilogram, BR, 

In [36]:
gasoline = [act for act in ei if 'petrol production, unleaded, petroleum refinery operation' in act['name'] and 'Europe without Switzerland' in act['location']][0]

In [42]:
for act in [act for act in ei if 'electricity, low voltage' in act['name'] and 'RER' in act['location']]:
    print(act)

'market group for electricity, low voltage' (kilowatt hour, RER, None)


In [43]:
electricity =  [act for act in ei if 'market group for electricity, low voltage' in act['name'] and 'RER' in act['location']][0]

In [53]:
EV_tot.new_exchange(input = electricity, amount = 0.113, type = 'technosphere', overwrite = True).save()
ICEV_tot.new_exchange(input = gasoline, amount =(2.48*0.023), type = 'technosphere').save()

In [54]:
for exc in EV_tot.exchanges():
    print(exc.amount)

6.666666666666667e-06
0.113
0.113


In [69]:
electricity.update(amount = 0.225)

In [310]:
database_dp = load_datapackage(ZipFS(ei.filepath_processed()))
method_dp = load_datapackage(ZipFS(bd.Method(method_GWP_recipe).filepath_processed()))

In [148]:
int_dp = bwp.create_datapackage()

In [335]:
indices_array = np.array([(steel.id,glider.id), (aluminium.id, glider.id)], dtype = bwp.INDICES_DTYPE)
indices_array

array([( 9736, 23558), (19241, 23558)],
      dtype=[('row', '<i4'), ('col', '<i4')])

In [336]:
int_dp.add_dynamic_vector(
    matrix = 'technosphere',
    interface = interf,
    name = 'Steel replacement',
    flip_array = np.array([True, True]),
    indices_array = indices_array,
)

NonUnique: This group name already used

In [282]:
method_GWP_recipe = [m for m in bd.methods if 'ReCiPe' in str(m) and  'Midpoint (H)' in str(m) and 'climate change' in str(m)][0]
fu = {EV_tot:1}

In [283]:
for act in [act for act in ei if 'Glider' in act['name']]:
    print(act)

'Glider' (unit, GLO, None)


In [284]:
fu

{'EV_tot' (unit/vkm, GLO, None): 1}

In [285]:
lca = bc.LCA(demand = fu, method = method_GWP_recipe)

In [286]:
lca.lci()
lca.lcia()

In [287]:
lca.score

0.1289011829468778

In [288]:
by_activity = np.array(lca.characterized_inventory.sum(axis = 0))
by_activity.shape

(1, 19133)

In [293]:
sorted_values = sorted([(abs(value), value, index) for index, value in enumerate(by_activity[0, :])], reverse=True)

In [294]:
for _, value, index in sorted_values[:5]:
    print(value, bd.get_activity(lca.dicts.activity.reversed[index]))


0.011361684353415195 'electricity production, lignite' (kilowatt hour, DE, None)
0.005341085055262461 'heat and power co-generation, hard coal' (kilowatt hour, PL, None)
0.0046093616744404655 'electricity production, hard coal' (kilowatt hour, DE, None)
0.004235943099572984 'electricity production, hard coal' (kilowatt hour, UA, None)
0.0042174674895082195 'electricity production, natural gas, conventional power plant' (kilowatt hour, GB, None)


In [302]:
bd.get_activity(lca.dicts.activity.reversed[index])


'electricity production, natural gas, conventional power plant' (kilowatt hour, GB, None)

In [312]:
fu

{'EV_tot' (unit/vkm, GLO, None): 1}

In [340]:
lca = bc.LCA(
    demand = {EV_tot.id: 1},
    data_objs = [database_dp,method_dp,int_dp],
    use_distributions = False,
    seed_override = None
)

lca.lci()
lca.lcia()

In [341]:
lca.score

0.1289011829468778

In [342]:
pd.DataFrame([
    {
        'score': lca.score, 
    } 
    for _, _ in zip(lca, range(10))
])

,score
0,0.128901
1,0.128901
2,0.128901
3,0.128901
4,0.128901
5,0.128901
6,0.128901
7,0.128901
8,0.128901
9,0.128901
